In [1]:
import numpy as np
import pandas as pd
import os,pickle, gc
from tqdm import tqdm
import matplotlib.pyplot as plt


In [2]:
pd.set_option("max_columns",300)
pd.set_option("max_rows",300)

In [3]:
train = pd.read_pickle("../pickle/train.pkl")

In [4]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [29]:
train_bond = pd.read_csv("../external/train_bonds.csv")

In [30]:
test_bond = pd.read_csv("../external/test_bonds.csv")

In [91]:
st = pd.read_pickle("../pickle/structures.pkl")

In [31]:
del train_bond["Unnamed: 0"], test_bond["Unnamed: 0"]

In [32]:
train_bond.head(20)

,molecule_name,atom_index_0,atom_index_1,nbond,L2dist,error,bond_type
0,dsgdb9nsd_000001,0,1,1,1.091953,0,1CH
1,dsgdb9nsd_000001,0,2,1,1.091952,0,1CH
2,dsgdb9nsd_000001,0,3,1,1.091946,0,1CH
3,dsgdb9nsd_000001,0,4,1,1.091948,0,1CH
4,dsgdb9nsd_000002,0,1,1,1.017190,0,1HN
5,dsgdb9nsd_000002,0,2,1,1.017187,0,1HN
6,dsgdb9nsd_000002,0,3,1,1.017208,0,1HN
7,dsgdb9nsd_000003,0,1,1,0.962107,0,1HO
8,dsgdb9nsd_000003,0,2,1,0.962107,0,1HO
9,dsgdb9nsd_000005,0,2,1,1.066598,0,1CH


In [33]:
train_bond["bond_type"].value_counts()

1CH    709416
1CC    439391
1CO    130279
1CN    107955
1HN     43363
2CO     34719
1HO     33058
2CC     26007
2CN     25263
3CC     12527
3CN     11327
1NN      5295
1NO      4337
1CF      1970
2NN      1269
2NO        18
Name: bond_type, dtype: int64

In [34]:
def replace_concat(df):
    df = pd.concat([
        df,
        df.rename(columns={"atom_index_0":"atom_index_1","atom_index_1":"atom_index_0"})
    ],axis=0).reset_index(drop=True)
    return df

In [35]:
train_bond = replace_concat(train_bond)

/Users/kyo/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


## 1CFと1CHは共存する？

In [ ]:
train_has_1CF = train_bond[train_bond["bond_type"].isin(["1CF"])][["molecule_name","atom_index_1"]].drop_duplicates()\
            .merge(train_bond, on=["molecule_name","atom_index_1"], how="left")

In [65]:
train_has_1CF.groupby(["molecule_name","atom_index_1"])\
            ["bond_type"].apply(lambda x: (x=="1CH").sum()).reset_index()["bond_type"].value_counts()

0    3352
Name: bond_type, dtype: int64

　存在しない

## 1JHCにCに対してのBond

In [114]:
train_1JHC = train_bond[train_bond["bond_type"].isin(["1CH"])][["molecule_name","atom_index_1"]].drop_duplicates()\
            .merge(train_bond, on=["molecule_name","atom_index_1"], how="left")

In [115]:
train_1JHC_gr = pd.DataFrame(train_1JHC.groupby(["molecule_name","atom_index_1"])["bond_type"].value_counts())
train_1JHC_gr = train_1JHC_gr.rename(columns={"bond_type":"other_bond"}).reset_index()

In [116]:
train_1JHC_gr = train_1JHC_gr.rename(columns={"atom_index_1":"atom_index"})\
            .merge(st[["molecule_name","atom_index","atom"]], on=["molecule_name","atom_index"], how="left")
train_1JHC_gr = train_1JHC_gr[train_1JHC_gr.atom=="C"]

In [119]:
train_1JHC_gr.head()

,molecule_name,atom_index,bond_type,other_bond,atom,bond_summary
0,dsgdb9nsd_000001,0,1CH,4,C,1CH4
5,dsgdb9nsd_000005,0,1CH,1,C,1CH1
6,dsgdb9nsd_000005,0,3CN,1,C,3CN1
8,dsgdb9nsd_000007,0,1CH,3,C,1CH3
9,dsgdb9nsd_000007,0,1CC,1,C,1CC1


In [125]:
train_1JHC_gr.sort_values(["molecule_name","atom_index","bond_type","other_bond"], inplace=True)
train_1JHC_gr["bond_summary"] = train_1JHC_gr["bond_type"] + train_1JHC_gr["other_bond"].astype(str)

In [126]:
train_1JHC_pattern = train_1JHC_gr \
        .groupby(["molecule_name","atom_index"])["bond_summary"].apply(lambda x: "_".join(x))

In [142]:
bond_sum = train_1JHC_pattern.str[0].astype(int) * train_1JHC_pattern.str[3].astype(int) \
+ train_1JHC_pattern.str[5].astype(float).fillna(0) * train_1JHC_pattern.str[8].astype(float).fillna(0) \
+ train_1JHC_pattern.str[10].astype(float).fillna(0) * train_1JHC_pattern.str[13].astype(float).fillna(0)

In [143]:
bond_sum.value_counts()

4.0    411937
3.0      1558
Name: bond_summary, dtype: int64

In [133]:
train_1JHC_pattern.value_counts()

1CC2_1CH2         89861
1CC1_1CH3         61266
1CC3_1CH1         55626
1CC2_1CH1_1CO1    43011
1CC1_1CH2_1CO1    34539
1CC2_1CH1_1CN1    28567
1CC1_1CH1_2CC1    23509
1CC1_1CH2_1CN1    22319
1CC1_1CH1_2CO1     9354
1CH1_3CC1          9205
1CH3_1CN1          7110
1CH3_1CO1          5756
1CH1_1CN1_2CC1     5692
1CC1_1CH1_2CN1     3314
1CH1_1CO1_2CN1     2981
1CH1_1CN1_2CN1     2732
1CH1_1CN1_2CO1     2257
1CH1_1CO1_2CC1     2002
1CC1_1CH1_1CO2     1085
1CH1_1CO1_2CO1      873
1CH2_1CO2           861
1CC1_1CH1_1CN1      450
1CC2_1CH1           340
1CC1_1CH1_1CO1      300
1CH1_1CN2           206
1CH1_1CN1_1CO1      152
1CH1_2CC1            55
1CC1_1CH2            51
1CH2_2CC1            15
1CH3                  4
1CH1_3CN1             1
1CH4                  1
Name: bond_summary, dtype: int64

In [173]:
train_1JHC_pattern[train_1JHC_pattern == "1CC1_1CH2"]

molecule_name     atom_index
dsgdb9nsd_000271  4             1CC1_1CH2
dsgdb9nsd_006033  3             1CC1_1CH2
dsgdb9nsd_007375  4             1CC1_1CH2
dsgdb9nsd_007654  5             1CC1_1CH2
dsgdb9nsd_010349  3             1CC1_1CH2
dsgdb9nsd_013833  9             1CC1_1CH2
dsgdb9nsd_014318  6             1CC1_1CH2
dsgdb9nsd_018308  6             1CC1_1CH2
dsgdb9nsd_028251  9             1CC1_1CH2
dsgdb9nsd_029540  6             1CC1_1CH2
dsgdb9nsd_030761  3             1CC1_1CH2
dsgdb9nsd_033400  10            1CC1_1CH2
dsgdb9nsd_042424  4             1CC1_1CH2
dsgdb9nsd_045541  6             1CC1_1CH2
dsgdb9nsd_045546  8             1CC1_1CH2
dsgdb9nsd_046611  7             1CC1_1CH2
dsgdb9nsd_050736  4             1CC1_1CH2
dsgdb9nsd_054124  3             1CC1_1CH2
dsgdb9nsd_054412  7             1CC1_1CH2
dsgdb9nsd_054611  3             1CC1_1CH2
dsgdb9nsd_054691  10            1CC1_1CH2
dsgdb9nsd_054692  9             1CC1_1CH2
dsgdb9nsd_055410  7             1CC1_1CH2
dsgdb

## atomをマージ

In [145]:
train_bond = train_bond.merge(st[["molecule_name","atom_index","atom"]], left_on=["molecule_name","atom_index_0"],
                              right_on=["molecule_name","atom_index"], how="left").rename(columns={"atom":"atom_0"})
del train_bond["atom_index"]
train_bond = train_bond.merge(st[["molecule_name","atom_index","atom"]], left_on=["molecule_name","atom_index_1"],
                              right_on=["molecule_name","atom_index"], how="left").rename(columns={"atom":"atom_1"})
del train_bond["atom_index"]

In [151]:
tmp = train_bond[train_bond.bond_type=="1CH"][["molecule_name","atom_index_1"]].drop_duplicates()\
    .merge(train_bond, on=["molecule_name","atom_index_1"], how="left")

In [155]:
tmp.sort_values(["molecule_name","atom_index_1","bond_type"], inplace=True)
bond_pattern = tmp[tmp.atom_1 == "C"].groupby(["molecule_name","atom_index_1"])["bond_type"].apply(lambda x: "_".join(x))

In [168]:
st.head(10)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602


In [167]:
train.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095


In [170]:
tmp.head(20)

,molecule_name,atom_index_1,L2dist,atom_index_0,bond_type,error,nbond,atom_0,atom_1
709416,dsgdb9nsd_000001,0,1.091953,1,1CH,0,1,H,C
709417,dsgdb9nsd_000001,0,1.091952,2,1CH,0,1,H,C
709418,dsgdb9nsd_000001,0,1.091946,3,1CH,0,1,H,C
709419,dsgdb9nsd_000001,0,1.091948,4,1CH,0,1,H,C
0,dsgdb9nsd_000001,1,1.091953,0,1CH,0,1,C,H
1,dsgdb9nsd_000001,2,1.091952,0,1CH,0,1,C,H
2,dsgdb9nsd_000001,3,1.091946,0,1CH,0,1,C,H
3,dsgdb9nsd_000001,4,1.091948,0,1CH,0,1,C,H
709420,dsgdb9nsd_000005,0,1.066598,2,1CH,0,1,H,C
709421,dsgdb9nsd_000005,0,1.151748,1,3CN,0,3,N,C


### H以外との結合

In [169]:
tmp[(tmp.atom_0!="H")&(tmp.atom_1=="C")]

,molecule_name,atom_index_1,L2dist,atom_index_0,bond_type,error,nbond,atom_0,atom_1
709421,dsgdb9nsd_000005,0,1.151748,1,3CN,0,3,N,C
709425,dsgdb9nsd_000007,0,1.529629,1,1CC,0,1,C,C
709426,dsgdb9nsd_000007,1,1.529629,0,1CC,0,1,C,C
709433,dsgdb9nsd_000008,0,1.415752,1,1CO,0,1,O,C
709437,dsgdb9nsd_000009,0,1.455009,1,1CC,0,1,C,C
709438,dsgdb9nsd_000009,2,1.201441,1,3CC,0,3,C,C
709443,dsgdb9nsd_000010,0,1.456930,1,1CC,0,1,C,C
709447,dsgdb9nsd_000011,0,1.507624,1,1CC,0,1,C,C
709448,dsgdb9nsd_000011,1,1.507624,0,1CC,0,1,C,C
709450,dsgdb9nsd_000011,1,1.203628,2,2CO,0,2,O,C


In [165]:
tmp[tmp.atom_0!="H"].groupby(["molecule_name","atom_index_1"])["L2dist"].mean().reset_index()

,molecule_name,atom_index_1,L2dist
0,dsgdb9nsd_000001,1,1.091953
1,dsgdb9nsd_000001,2,1.091952
2,dsgdb9nsd_000001,3,1.091946
3,dsgdb9nsd_000001,4,1.091948
4,dsgdb9nsd_000005,0,1.151748
5,dsgdb9nsd_000005,2,1.066598
6,dsgdb9nsd_000007,0,1.529629
7,dsgdb9nsd_000007,1,1.529629
8,dsgdb9nsd_000007,2,1.094954
9,dsgdb9nsd_000007,3,1.094958


In [156]:
bond_pattern.reset_index().head(10)

,molecule_name,atom_index_1,bond_type
0,dsgdb9nsd_000001,0,1CH_1CH_1CH_1CH
1,dsgdb9nsd_000005,0,1CH_3CN
2,dsgdb9nsd_000007,0,1CC_1CH_1CH_1CH
3,dsgdb9nsd_000007,1,1CC_1CH_1CH_1CH
4,dsgdb9nsd_000008,0,1CH_1CH_1CH_1CO
5,dsgdb9nsd_000009,0,1CC_1CH_1CH_1CH
6,dsgdb9nsd_000009,2,1CH_3CC
7,dsgdb9nsd_000010,0,1CC_1CH_1CH_1CH
8,dsgdb9nsd_000011,0,1CC_1CH_1CH_1CH
9,dsgdb9nsd_000011,1,1CC_1CH_2CO


In [157]:
target = train.merge(bond_pattern, on=["molecule_name","atom_index_1"], how="left")

In [158]:
target.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,bond_type
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,1CH_1CH_1CH_1CH
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,NaN
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,NaN
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,NaN
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1CH_1CH_1CH_1CH


In [160]:
target.groupby("bond_type")["scalar_coupling_constant"].mean().sort_values()

bond_type
1CC_1CC_1CC_1CH     12.905424
1CC_1CC_1CH_1CO     16.439438
1CC_1CC_1CH_1CN     17.025175
1CC_1CH_1CO_1CO     17.074869
1CC_1CC_1CH         24.032786
1CC_1CH_2CC         24.538462
1CC_1CH_1CH         26.330470
1CC_1CC_1CH_1CH     26.548853
1CH_1CH_2CC         27.152548
1CC_1CH_2CO         31.796474
1CC_1CH_1CH_1CO     34.252631
1CC_1CH_1CH_1CN     34.924141
1CH_1CN_2CC         38.517565
1CC_1CH_1CN         38.703342
1CH_1CN_2CO         39.696448
1CC_1CH_1CH_1CH     44.616523
1CH_1CN_1CN         46.644783
1CH_1CO_2CN         47.375839
1CH_1CN_2CN         49.170171
1CH_1CH_1CH         49.614739
1CH_1CH_1CO_1CO     51.571643
1CC_1CH_2CN         55.517006
1CH_1CO_2CC         56.215762
1CC_1CH_1CO         56.911977
1CH_1CH_1CH_1CN     60.342317
1CH_1CH_1CH_1CO     70.466938
1CH_1CO_2CO         72.187712
1CH_1CH_1CH_1CH     84.808450
1CH_1CN_1CO        104.264039
1CH_3CC            113.371318
1CH_3CN            171.220000
1CH_2CC            196.945618
Name: scalar_coupling_constant

In [172]:
bond_pattern[bond_pattern == "1CH3"]

Series([], Name: bond_type, dtype: object)